In [1]:
import sys
from awsglue.utils import getResolvedOptions
import pyspark.sql.functions as F
from pyspark import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from pyspark.sql.functions import col, max



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:

# from helpers import get_glue_env_var, get_latest_partitions, PARTITION_KEYS
# from helpers import get_glue_env_var, PARTITION_KEYS

glueContext = GlueContext(SparkContext.getOrCreate())
job = Job(glueContext)

PARTITION_KEYS = ["import_year", "import_month", "import_day", "import_date"]

# def get_latest_partitions(dfa):
#    dfa = dfa.where(col('import_year') == dfa.select(max('import_year')).first()[0])
#    dfa = dfa.where(col('import_month') == dfa.select(max('import_month')).first()[0])
#    dfa = dfa.where(col('import_day') == dfa.select(max('import_day')).first()[0])
#    return dfa

def add_import_time_columns(data_frame):
    now = datetime.datetime.now()
    importYear = str(now.year)
    importMonth = str(now.month).zfill(2)
    importDay = str(now.day).zfill(2)
    importDate = importYear + importMonth + importDay

    data_frame = data_frame.withColumn(
        'import_datetime', f.current_timestamp())
    data_frame = data_frame.withColumn(
        'import_timestamp', f.lit(str(now.timestamp())))
    data_frame = data_frame.withColumn('import_year', f.lit(importYear))
    data_frame = data_frame.withColumn('import_month', f.lit(importMonth))
    data_frame = data_frame.withColumn('import_day', f.lit(importDay))
    data_frame = data_frame.withColumn('import_date', f.lit(importDate))
    return data_frame



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
tableList = ["applications"]
# tableList = ["public_comments"]
databaseName = "dataplatform-stg-tascomi-refined-zone"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
daysBuffer = 5
if daysBuffer > 0:
    pushDownPredicate = f"import_date>=date_format(date_sub(current_date, {daysBuffer}), 'yyyyMMdd')"
else:
    pushDownPredicate = ""


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
for nameOfTableToRecast in tableList:
    
#   load data
    source_ddf = glueContext.create_dynamic_frame.from_catalog(
        name_space=databaseName,
        table_name=nameOfTableToRecast,
#         push_down_predicate = pushDownPredicate
        push_down_predicate = "import_date = '20211108'"
    )

    source_df = source_ddf.toDF()
    
    print((source_df.count()))


#     source_df = get_latest_partitions(source_df)

#   remove json col
#     source_df = source_df.drop("json")
#     source_df = source_df.drop(nameOfTableToRecast)

#   recast
    





FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [18]:
## WRITE snapshot TO S3
nameOfTableToRecast = 'applications'
def add_currency_date_columns(data_frame, currencyDateString):
    currencyYear = currencyDateString[0:4]
    currencyMonth = currencyDateString[4:6]
    currencyDay = currencyDateString[6:8]
    currencyDate = currencyDateString
    data_frame = data_frame.withColumn('currency_year', F.lit(currencyYear))
    data_frame = data_frame.withColumn('currency_month', F.lit(currencyMonth))
    data_frame = data_frame.withColumn('currency_day', F.lit(currencyDay))
    data_frame = data_frame.withColumn('currency_date', F.lit(currencyDate))
    return data_frame


PARTITION_KEYS = ["currency_year", "currency_month", "currency_day", "currency_date"]
source_df = add_currency_date_columns(source_df, '20211030')

resultDataFrame = DynamicFrame.fromDF(source_df, glueContext, "resultDataFrame")

target_destination = f"s3://dataplatform-stg-refined-zone/TEMP/snapshot/{nameOfTableToRecast}"
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=resultDataFrame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_destination, "partitionKeys": PARTITION_KEYS},
    transformation_ctx="parquetData")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
## WRITE increment TO S3

resultDataFrame = DynamicFrame.fromDF(source_df, glueContext, "resultDataFrame")
target_destination = f"s3://dataplatform-stg-refined-zone/TEMP/increment/{nameOfTableToRecast}"

PARTITION_KEYS = ["import_year", "import_month", "import_day", "import_date"]

parquetData = glueContext.write_dynamic_frame.from_options(
    frame=resultDataFrame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_destination, "partitionKeys": PARTITION_KEYS},
    transformation_ctx="parquetData")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Run crawler on increments now!

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
daysBuffer = 5
nameOfTableToRecast = 'applications'
def get_latest_snapshot(dfa):
   dfa = dfa.where(col('currency_date') == dfa.select(max('currency_date')).first()[0])
   return dfa


# load 5 latest snapshots and only keep the last one
snapshot_ddf = glueContext.create_dynamic_frame.from_catalog(
        name_space="dataplatform-stg-tascomi-refined-zone",
        table_name=f"snapshot_{nameOfTableToRecast}",
        push_down_predicate = f"currency_date>=date_format(date_sub(current_date, {daysBuffer}), 'yyyyMMdd')"
    )

snapshot_df = snapshot_ddf.toDF()
snapshot_df = get_latest_snapshot(snapshot_df)
last_currency_date = snapshot_df.select(max('currency_date')).first()[0]
print(snapshot_df.count())

print(last_currency_date)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

72207
20211106

In [7]:
# load increments later than the currency date
increment_ddf = glueContext.create_dynamic_frame.from_catalog(
        name_space="dataplatform-stg-tascomi-refined-zone",
        table_name=f"increment_{nameOfTableToRecast}",
        push_down_predicate = f"import_date>{last_currency_date}"
    )
increment_df = increment_ddf.toDF()
print(increment_df.count())




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [8]:
# In case there are several days worth of increments: only keep the latest version of a record
from pyspark.sql import Window
w = Window.partitionBy('id')
increment_df = increment_df.withColumn('latest', F.max('last_updated').over(w))\
    .where(F.col('last_updated') == F.col('latest'))\
    .drop('latest')
print(increment_df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [9]:
new_currency_date = increment_df.select(max('import_date')).first()[0]

print('new currency date = ' + new_currency_date)
# apply COU
snapshot_df = snapshot_df.join(increment_df, 'id', 'left_anti')
print('size of snapshot minus increment')
print(snapshot_df.count())



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

new currency date = 20211108
size of snapshot minus increment
72205

In [10]:
def add_currency_date_columns(data_frame, currencyDateString):
    currencyYear = currencyDateString[0:4]
    currencyMonth = currencyDateString[4:6]
    currencyDay = currencyDateString[6:8]
    currencyDate = currencyDateString
    data_frame = data_frame.withColumn('currency_year', F.lit(currencyYear))
    data_frame = data_frame.withColumn('currency_month', F.lit(currencyMonth))
    data_frame = data_frame.withColumn('currency_day', F.lit(currencyDay))
    data_frame = data_frame.withColumn('currency_date', F.lit(currencyDate))
    return data_frame

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
increment_df = add_currency_date_columns(increment_df,new_currency_date)

snapshot_df = snapshot_df.unionByName(increment_df)
print('size of snapshot merged with increment = ')
print(snapshot_df.count())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

size of snapshot merged with increment = 
72207

In [12]:
snapshot_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: integer (nullable = true)
 |-- a_charitable_use: boolean (nullable = true)
 |-- a_interior_works: boolean (nullable = true)
 |-- a_new_build_floorspace_100m2: boolean (nullable = true)
 |-- a_new_build_permitted_floorspace_100m2: boolean (nullable = true)
 |-- a_new_building_or_operations_proposed: boolean (nullable = true)
 |-- a_new_residential_floorspace: boolean (nullable = true)
 |-- a_rear_projection_m: double (nullable = true)
 |-- a_total_demolition: boolean (nullable = true)
 |-- access_hardstand_existing: string (nullable = true)
 |-- access_hardstand_existing_details: string (nullable = true)
 |-- access_hardstand_proposed: string (nullable = true)
 |-- access_hardstand_proposed_details: string (nullable = true)
 |-- acrylonitrile_tonnes_: double (nullable = true)
 |-- activities_processes_description: string (nullable = true)
 |-- admin_officer_id: integer (nullable = true)
 |-- advert_already_in_place: boolean (nullable = true)
 |-- advert_description: string

In [13]:
increment_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- a_charitable_use: boolean (nullable = true)
 |-- a_interior_works: boolean (nullable = true)
 |-- a_new_build_floorspace_100m2: boolean (nullable = true)
 |-- a_new_build_permitted_floorspace_100m2: boolean (nullable = true)
 |-- a_new_building_or_operations_proposed: boolean (nullable = true)
 |-- a_new_residential_floorspace: boolean (nullable = true)
 |-- a_rear_projection_m: double (nullable = true)
 |-- a_total_demolition: boolean (nullable = true)
 |-- access_hardstand_existing: string (nullable = true)
 |-- access_hardstand_existing_details: string (nullable = true)
 |-- access_hardstand_proposed: string (nullable = true)
 |-- access_hardstand_proposed_details: string (nullable = true)
 |-- acrylonitrile_tonnes_: double (nullable = true)
 |-- activities_processes_description: string (nullable = true)
 |-- admin_officer_id: integer (nullable = true)
 |-- advert_already_in_place: boolean (nullable = true)
 |-- advert_description: string (nullable = true)
 |-- advert_deta

In [14]:
# add currency date and set it a partition key
snapshot_df = add_currency_date_columns(snapshot_df, new_currency_date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
snapshot_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------------+----------------+----------------------------+--------------------------------------+-------------------------------------+----------------------------+-------------------+------------------+-------------------------+---------------------------------+-------------------------+---------------------------------+---------------------+--------------------------------+----------------+-----------------------+------------------+----------------------------+-------------------------+-------------------------------------------+-------------------+------------------+----------------+------------------+--------------------------------+----------------------------+--------+-----------------+-------------------------------------------+--------------------------------------------+---------------------------------------+---------------------------------------+---------------------+----------------------------+---------------------------+----------------------+-----------------

In [16]:
## WRITE snapshot TO S3

PARTITION_KEYS = ["currency_year", "currency_month", "currency_day", "currency_date"]


resultDataFrame = DynamicFrame.fromDF(snapshot_df, glueContext, "resultDataFrame")

target_destination = f"s3://dataplatform-stg-refined-zone/TEMP/snapshot/{nameOfTableToRecast}"
parquetData = glueContext.write_dynamic_frame.from_options(
    frame=resultDataFrame,
    connection_type="s3",
    format="parquet",
    connection_options={"path": target_destination, "partitionKeys": PARTITION_KEYS},
    transformation_ctx="parquetData")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…